<a href="https://colab.research.google.com/github/Anchayadav/AutoTS/blob/main/RFM_Analysis_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import  pandas as pd
import plotly.express as px
import plotly.io as pio
pio.templates.default='plotly_white'
import plotly.graph_objects as go
data=pd.read_csv('rfm_data.csv')
print(data.head())

   CustomerID PurchaseDate  TransactionAmount ProductInformation  OrderID  \
0        8814   2023-04-11             943.31          Product C   890075   
1        2188   2023-04-11             463.70          Product A   176819   
2        4608   2023-04-11              80.28          Product A   340062   
3        2559   2023-04-11             221.29          Product A   239145   
4        9482   2023-04-11             739.56          Product A   194545   

   Location  
0     Tokyo  
1    London  
2  New York  
3    London  
4     Paris  


In [ ]:
from datetime import datetime
data['PurchaseDate']=pd.to_datetime(data['PurchaseDate'])
data['Recency'] = (datetime.now().date() - data['PurchaseDate'].dt.date).dt.days

frequency_data=data.groupby('CustomerID')['OrderID'].count().reset_index()
frequency_data.rename(columns={'OrderID':'Frequency'},inplace=True)
data=data.merge(frequency_data, on='CustomerID',how='left')

monetary_data =data.groupby('CustomerID')['TransactionAmount'].sum().reset_index()
monetary_data.rename(columns={'TransactionAmount':'MonetaryValue'},inplace=True)
data=data.merge(monetary_data,on='CustomerID',how='left')

In [ ]:
print(data.head())

   CustomerID PurchaseDate  TransactionAmount ProductInformation  OrderID  \
0        8814   2023-04-11             943.31          Product C   890075   
1        2188   2023-04-11             463.70          Product A   176819   
2        4608   2023-04-11              80.28          Product A   340062   
3        2559   2023-04-11             221.29          Product A   239145   
4        9482   2023-04-11             739.56          Product A   194545   

   Location  Recency  Frequency  MonetaryValue  
0     Tokyo      155          1         943.31  
1    London      155          1         463.70  
2  New York      155          1          80.28  
3    London      155          1         221.29  
4     Paris      155          1         739.56  


In [ ]:
recency_score=[5,4,3,2,1]
frequency_score=[1,2,3,4,5]
monetary_score=[1,2,3,4,5]

data['RecencyScore']=pd.cut(data['Recency'],bins=5,labels=recency_score)
data['FrequencyScore']=pd.cut(data['Frequency'],bins=5,labels=frequency_score)
data['MonetaryScore']=pd.cut(data['MonetaryValue'],bins=5,labels=monetary_score)


In [ ]:
data['RecencyScore']=data['RecencyScore'].astype('int')
data['FrequencyScore']=data['FrequencyScore'].astype('int')
data['MonetaryScore']=data['MonetaryScore'].astype('int')

In [ ]:
data['RFM_Score'] = data['RecencyScore']+data['FrequencyScore']+data['MonetaryScore']
segment_label=['low-Value','Mid-Value','High-Value']
data['Value Segment'] =pd.qcut(data['RFM_Score'],q=3,labels=segment_label)

In [ ]:
print(data.head())

In [ ]:
segment_counts=data['Value Segment'].value_counts().reset_index()
segment_counts.columns=['Value Segment','Count']

pastel_colors = px.colors.qualitative.Pastel

fig_segment_dist =px.bar(segment_counts,x='Value Segment',y='Count',
                         color='Value Segment',color_discrete_sequence=pastel_colors,
                         title='RFM Value Segent Distribution')

fig_segment_dist.update_layout(xaxis_title='RFM Value Segment',
                               yaxis_title='Count',
                              showlegend=False)
fig_segment_dist.show()

In [ ]:
data['RFM Customer Segment'] =''
data.loc[data['RFM_Score'] > 9,'RFM Customer Segment'] ='Champions'
data.loc[(data['RFM_Score'] >= 6) & (data['RFM_Score']< 9),'RFM Customer Segment'] ='Potential Loyalists'
data.loc[(data['RFM_Score'] >= 5) & (data['RFM_Score']< 6 ),'RFM Customer Segment'] ='At Risk Customer'
data.loc[(data['RFM_Score'] >= 4) & (data['RFM_Score']< 4 ),'RFM Customer Segment'] ="Can't lose"
data.loc[(data['RFM_Score'] >= 3) & (data['RFM_Score']< 3),'RFM Customer Segment'] ='lost'

print(data[['CustomerID','RFM Customer Segment']])

     CustomerID RFM Customer Segment
0          8814                     
1          2188                     
2          4608                     
3          2559                     
4          9482                     
..          ...                  ...
995        2970  Potential Loyalists
996        6669  Potential Loyalists
997        8836  Potential Loyalists
998        1440  Potential Loyalists
999        4759  Potential Loyalists

[1000 rows x 2 columns]


In [ ]:
champions_segment=data[data['RFM Customer Segment']=='Champions']
fig=go.Figure()
fig.add_trace(go.Box(y=champions_segment['RecencyScore'],name='Recency'))
fig.add_trace(go.Box(y=champions_segment['FrequencyScore'],name='Frequency'))
fig.add_trace(go.Box(y=champions_segment['MonetaryScore'],name='Monetary'))

fig.update_layout(title='Distribution of RFM Value within Champions SEgment',
                  yaxis_title='RFM Value',
                  showlegend=True)
fig.show()


In [ ]:
correlation_matrix=champions_segment[['RecencyScore','FrequencyScore','MonetaryScore']].corr()
fig_heatmap=go.Figure(data=go.Heatmap(
                 z=correlation_matrix.values,
                 x=correlation_matrix.columns,
                 y=correlation_matrix.columns,
                 colorscale='RdBu',
                 colorbar=dict(title='Correlation')))

fig.update_layout(title='Correlation Mtrix of RFM  Vlaues within Champions  Seegment')
fig.show()


0      1
1      1
2      1
3      1
4      1
      ..
995    5
996    5
997    5
998    5
999    5
Name: RecencyScore, Length: 1000, dtype: category
Categories (5, int64): [5 < 4 < 3 < 2 < 1]